In [ ]:
from src import NeuralTester, ExperimentConfig
import torch
from src.learner import PymooLearner
from src.learner.auxiliary_components import PYMOO_NSGA2_DEFAULT_PARAMS
from src.objective_functions import uqi, confidence_balance
from models import load_stylegan
import logging
import wandb
import sys

## Example usage of the Neural Tester
We use wandb to log progress in various steps!

In [ ]:
wandb.login()

In [ ]:
# Setup logging to be able to view logging outputs in notebooks.
logging.basicConfig(
    level=logging.INFO, format="[%(asctime)s - %(name)s - %(levelname)s] %(message)s"
)
logger = logging.getLogger()
logger.handlers[0].stream = sys.stdout

In [ ]:
# Define the configurations for our experiments.
conf = ExperimentConfig(
    samples_per_class=10,
    generations=50,
    mix_dim_range=(0, 8),
    predictor="../models/wrn_mnist.pkl",
    generator="../models/sg2_mnist.pkl",
    learner=PymooLearner,
    metric=uqi,
)

learner_params = PYMOO_NSGA2_DEFAULT_PARAMS
learner_params["n_var"] = conf.genome_size

In [ ]:
predictor = torch.load(conf.predictor)  # The System under test (SUT)
generator = load_stylegan(conf.generator)  # The generator network (a stylegan in this case)
learner = conf.learner(**learner_params)  # The learner for search based optimization of candidates.
device = torch.device("cuda")  # The target device for all operaitons.

In [ ]:
# MOO Objective functions.
objective_functions = [uqi, confidence_balance]

In [ ]:
# Here we initialize the Tester object.
tester = NeuralTester(
    predictor=predictor,
    generator=generator,
    learner=learner,
    objective_functions=objective_functions,
    num_generations=conf.generations,
    mix_dim_range=conf.mix_dim_range,
    device=device,
)

In [ ]:
# We start the testing procedure.
tester.test(
    num_classes=10,
    samples_per_class=conf.samples_per_class,
)